In [ ]:
import torch
import cvzone
import cv2
from torchvision import transforms
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
import os
from PIL import Image


model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
model.eval()  


classnames_path = r'C:\Users\KAARTHI\Downloads\CodEvo Solutions\classes.txt'
if os.path.exists(classnames_path):
    with open(classnames_path, 'r') as f:
        classnames = f.read().splitlines()
else:
    print("Classnames file does not exist.")
    classnames = []


image_path = r'C:\Users\KAARTHI\Downloads\CodEvo Solutions\dog.jpg'
if os.path.exists(image_path):
    print("Image found!")

    
    image = cv2.imread(image_path)
    if image is None:
        print("Failed to load image.")
    else:
        print("Image loaded successfully.")
        
        image_resized = cv2.resize(image, (640, 480))
        image_transform = transforms.ToTensor()
        img = image_transform(image_resized)

        with torch.no_grad():
            pred = model([img])
            print(pred)  
            bbox, scores, labels = pred[0]['boxes'], pred[0]['scores'], pred[0]['labels']
            confidence_threshold = 0.70
            conf = torch.argwhere(scores > confidence_threshold).shape[0]
            print(f"Objects detected: {conf}")

            for i in range(conf):
                x, y, w, h = bbox[i].numpy().astype('int')
                classname = labels[i].numpy().astype('int')
                if classname < len(classnames):
                    class_detected = classnames[classname]
                else:
                    class_detected = "Unknown"

                cv2.rectangle(image_resized, (x, y), (w, h), (0, 0, 255), 4)
                cvzone.putTextRect(image_resized, class_detected, [x + 8, y - 12], scale=2, border=2)
        image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
        output_path = 'output_with_detection.png'
        cv2.imwrite(output_path, image_resized)
        print(f"Image saved to {output_path}")
        cv2.imshow('Detected Image', image_resized)
        cv2.waitKey(0)
        cv2.destroyAllWindows()  

else:
    print("Image file does not exist.")


Image found!
Image loaded successfully.
[{'boxes': tensor([[229.2657,  54.3510, 403.9397, 444.4980],
        [194.8293, 406.1061, 206.5419, 422.1459],
        [521.8044, 379.5511, 552.8061, 390.4660],
        [242.0619, 178.3920, 402.9124, 443.9230],
        [303.5885, 237.3669, 328.8073, 286.5322],
        [459.9107, 361.8687, 480.3815, 372.1587],
        [418.9500, 319.7704, 422.9830, 327.8223],
        [303.7210, 242.6300, 328.9188, 288.4164],
        [195.6283, 406.6662, 206.3537, 420.9292],
        [112.8798, 408.4431, 144.6919, 426.8237],
        [522.3503, 379.7317, 552.6393, 390.1768],
        [121.7899, 309.4623, 136.7342, 325.9331]]), 'labels': tensor([18, 44, 16, 23, 44, 16,  1, 32, 47, 16, 44, 16]), 'scores': tensor([0.9959, 0.4617, 0.3756, 0.1870, 0.1557, 0.1285, 0.1272, 0.1114, 0.0852,
        0.0767, 0.0754, 0.0670])}]
Objects detected: 1
Image saved to output_with_detection.png
